# Gaussian fitting and mass fucntion calculations
This notebook creates density histograms and then fits a gaussian curve to them, as well as a skewnormal distribution. It then goes on to look at calculation of mass functions from a number of sourcesand visualise these. 

In [ ]:
#Import packages
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import scipy.stats
import scipy.optimize as scopt
from scipy.stats import norm
from scipy.stats import skew
from scipy.stats import skewnorm
from scipy.optimize import curve_fit as scurv
from scipy import integrate 

## Load data for gaussian plotting

In [ ]:
def load_data(rad):
    z = []
    deltas = []

    q = open(f'Text_data/z_{rad}.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        z.append(i.split(  )[0])
    q.close()

    z_arr = np.array(z).astype(float)
    sort = np.argsort(z_arr)
    z_arr = z_arr[sort]

    k = open(f'Text_Data/delta_{rad}.txt', 'r')
    lines = k.readlines()
    for i in lines:
        deltas.append(i.split(  ))
    k.close()

    delta_arr = np.array(deltas).astype(float)
    delta_arr = delta_arr[sort]
    
    return delta_arr, z_arr

## Load data, additionally loading mass data

In [ ]:
def load_mass_data(rad):
    
    z = []
    deltas = []
    masses = []

    q = open(f'Text_data/Mass_Calcs/z_{rad}_mass.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        z.append(i.split(  )[0])
    q.close()

    z_arr = np.array(z).astype(float)
    sort = np.argsort(z_arr)
    z_arr = z_arr[sort]

    k = open(f'Text_Data/Mass_Calcs/delta_{rad}_mass.txt', 'r')
    lines = k.readlines()
    for i in lines:
        deltas.append(i.split(  ))
    k.close()

    delta_arr = np.array(deltas).astype(float)
    delta_arr = delta_arr[sort]

    q = open(f'Text_data/Mass_Calcs/masses_{rad}.txt', 'r')
    
    lines = q.readlines()
    for i in lines: 
        masses.append(i.split(  )[0])
    q.close()
    
    mass_arr = np.array(masses).astype(float)
    mass_arr = mass_arr[sort]
   
    
    return delta_arr, z_arr, mass_arr

## Load halo virial masses as an array 

In [ ]:
def load_halos(filename):
    
    halo_mass =[]
    q = open(f'Text_data/{filename}', 'r')
    lines = q.readlines()
    for i in lines: 
        halo_mass.append(i.split(  )[0])
    q.close()
    
    halo_mass_arr = np.array(halo_mass).astype(float)
    return halo_mass_arr

## Define gaussian curve for fitting

In [ ]:
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

## Plot histograms and fit gaussian and skewnormal curves before returning arrays of std dev and skewness

In [ ]:
def gaussian_plot(z_arr, delta_arr, rad):
    
    var_arr = np.ones(len(z_arr))
    skew_arr = np.ones(len(z_arr))
    skew_err = np.ones(len(z_arr))
    x = np.linspace(-1,1,10000)
    ran = (-1, 1)
    for i in range(len(z_arr)):

        z = z_arr[i] 

        mu, std = norm.fit(delta_arr[i])
        z = np.round(z, decimals = 2)

        n_bin = 1000

        bin_heights, bin_borders = np.histogram(delta_arr[i], bins = n_bin, range = ran)
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 

        plt.clf()
        hist =plt.hist(delta_arr[i], bins = n_bin, range = ran, label = 'Overdensities', color = 'g' )
        plt.title(f'Overdensity Histogram for z = {z}')
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
        plt.savefig(f'Hists/Overdensity_{rad}Mpc_redshift_{z}.jpeg', dpi =200)

        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 40000)
        std = popt[2]
        var = std*std
        var_arr[i] *= var

        plt.plot(x, gaussian(x, *popt), label = f'Gaussian Fit', color = 'b' )
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
        plt.xlim(ran)
        plt.title(f'Expected Distribution of Overdensity Values for z = {z}')
        plt.legend()
        plt.savefig(f'Gaussians/Gaussian_{rad}Mpc_redshift_{z}.jpeg', dpi =200)

        skew_arr[i] *= skew((delta_arr[i]))
        data = skewnorm.fit(delta_arr[i])

        plt.clf()
        n_bin = 100
        plt.figure(figsize=(8,3))
        hist = plt.hist(delta_arr[i], bins = n_bin, label = 'Overdensities', range = ran, color = 'g')
        #plt.title(f'Overdensity Histogram for z = {z}')
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
        popt, pcov = scurv(skewnorm.pdf, bin_centers, bin_heights, p0 = [data[0], data[1], data[2]], maxfev = 50000)
        perr = np.sqrt(np.diag(pcov))
        
        a, loc, scale = skewnorm.fit(delta_arr[i])

        # Calculate the standard error of the skewness parameter
        n = len(delta_arr[i])
        s = np.std(data, ddof=1)
        m2 = np.mean((data - np.mean(delta_arr[i]))**2)
        m3 = np.mean((data - np.mean(delta_arr[i]))**3)
        SE_a = np.sqrt(6 * n * (n - 1) / ((n - 2) * (n + 1) * (n + 3))) * (m3 / s**3 - 3 * a * m2 / s**2 + 2 * a**3)
        
        skew_err[i] *= SE_a
        #skew_err[i] *= perr[0]
        
        width = hist[1][1] - hist[1][0]
        scale = len(delta_arr[i]) * width 
        plt.plot(x, scale*skewnorm.pdf(x, data[0], data[1], data[2] ), color = 'r', label = 'Skewnormal Fit')
        plt.savefig(f'Skewnormal/Skewnormal_{rad}Mpc_redshift_{z}.jpeg', dpi =200, bbox_inches = "tight")
     
    return var_arr, skew_arr, skew_err

## Subplots for use in report and presentation 

In [ ]:
def subplots_gaussian(z_arr, delta_arr, rad):
    
    ran = (-1,1)
    x = np.linspace(-1, 1, 1000)

    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(figsize = (8,4), nrows=2, ncols=2, sharex = True)
    
    z = np.round(z_arr, decimals = 2)
    
    bin_heights, bin_borders = np.histogram(delta_arr[0], bins = 30 , range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax4.hist(delta_arr[0], bins = 30, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 40000)
    ax4.plot(x, gaussian(x, *popt), color = 'r' )
    ax4.text(0.7,0.9, 'z = 0.0', transform=ax4.transAxes)
    
    
    bin_heights, bin_borders = np.histogram(delta_arr[5], bins = 50, range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax3.hist(delta_arr[5], bins = 50, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 40000)
    ax3.plot(x, gaussian(x, *popt), color = 'r' )
    ax3.text(0.7,0.9, f'z = {z[5]}', transform=ax3.transAxes)
    
    bin_heights, bin_borders = np.histogram(delta_arr[10], bins = 120, range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax2.hist(delta_arr[10], bins = 120, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 40000)
    ax2.plot(x, gaussian(x, *popt), color = 'r' )
    ax2.text(0.7,0.9, f'z = {z[10]}', transform=ax2.transAxes)
    
    bin_heights, bin_borders = np.histogram(delta_arr[-1], bins = 500, range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax1.hist(delta_arr[-1], bins = 500, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 40000)
    ax1.plot(x, gaussian(x, *popt), label = f'Fitted Curve', color = 'r' )
    ax1.text(0.7,0.9, 'z = 99.0', transform=ax1.transAxes)
    
    fig.text(0.5, 0.02, 'Overdensity $\delta$', ha='center', fontsize=14)
    fig.text(0.04, 0.5, 'Frequency', va='center', rotation='vertical', fontsize=14)
    fig.subplots_adjust(wspace=0.15, hspace=0.15)
    fig.legend(bbox_to_anchor = (0.595, 0))
    plt.savefig('Subplots\Gaussian_Subplots', dpi = 200)  

##  Numerical integration mass function method - finding area of curve 

In [ ]:
def area_beneath(delta_arr, z_arr):
    
    F = []
    sigma = []
    k = np.linspace(-1,2, 1000)
    for i in range(len(z_arr)):
    
        n_bin = 1000
        bin_heights, bin_borders = np.histogram(delta_arr[i], bins = n_bin, range = (-1,3))
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [np.mean(delta_arr[i]), 1, 0.05], maxfev = 50000)
        x, y = 1.686, np.inf

        area, error = integrate.quad(gaussian, x, y, args =(popt[0], popt[1], popt[2]))
        F.append(area)
        sigma.append(np.abs(popt[2]))

        
    return F, sigma

## Calculation of mass function from above method

In [ ]:
def numerical(F, rad, masses, rho_bar):
    
    dFdr = np.abs(np.gradient(F, rad))
    
    drdM = np.abs(np.gradient(rad, masses))
    
    dFdM = dFdr * drdM
    
    dndM = dFdM * (rho_bar/masses)
    
    return dndM

## Calculation of mass function by analytic method

In [ ]:
def press_schechter(d_c, sigma, masses, rho_bar):
    
    ps = np.sqrt(2/np.pi) * (d_c/sigma**2) * (rho_bar/masses) * np.abs(np.gradient((sigma), (masses))) * np.exp(-(d_c**2) /(2*sigma**2)) 
        
    return ps

## Write out gaussian fitting data

In [ ]:
def data_out(rad, var_arr, skew_arr, skew_err):
    outfile = open(f'Text_Data/variance_{rad}Mpc.txt', 'w')

    for i in range(len(var_arr)):
        outfile.write(str(var_arr[i]) + "\n")

    outfile.close()

    outfile = open(f'Text_Data/skew_{rad}Mpc.txt', 'w')

    for i in range(len(skew_arr)):
        outfile.write(str(skew_arr[i]) + "\n")

    outfile.close()
    
    outfile = open(f'Text_Data/skew_err_{rad}Mpc.txt', 'w')

    for i in range(len(skew_err)):
        outfile.write(str(skew_err[i]) + "\n")

    outfile.close()

## Plot mass functions using numerical and analytic methods

In [ ]:
def mass_funcs_plot(sigma_0, mass_0, F, rho_bar_0, rads):
    
    ps = press_schechter(1.686, sigma_0, mass_0, rho_bar_0)
    dndm = numerical(F, rads, mass_0, rho_bar_0)
    plt.plot((mass_0), ps, label = 'Analytic', color = 'g')
    plt.plot((mass_0), dndm, label = 'Numerical', color = 'k')
    plt.xlabel('$log_{10}({M}/{M_{sun}}$)')
    plt.ylabel('$\phi = dn/dlog_{10}M$  $(Mpc/h)^{-3}$')
    plt.yscale('log')
    #plt.xlim(12.4,15.7)
    plt.legend()
    plt.savefig('Numerical_vs_Analytic', dpi = 200)
    plt.show()
    plt.clf() 
    
    return ps, dndm

## Calculation of data for mass function plotting 

In [ ]:
def mass_funcs(rads):
    
    sigma_0 = []
    
    F_0,F_2,F_99 = [], [], []
    mass_0,mass_2,mass_99 = [], [], []
    for i in range(len(rads)):
        delta_arr, z_arr, mass_arr= load_mass_data(rads[i])
        mass_arr = mass_arr/(1.9884e33)
        F, sigmas = area_beneath(delta_arr, z_arr)
        sigma_0.append(sigmas[0]) 
        F_0.append(F[0]) 
        mass_0.append(mass_arr[0])  
       
    sigma_0 = np.array(sigma_0)
    mass_0 = np.array(mass_0)
    a = 1/(1+z_arr)
    
    rho_bar_0 = (mass_0[0]*0.695**3)/((4*np.pi*1**3)/3)
    rho_bar_0 = ((3*(0.695*100)**2)/(8*np.pi*4.7e-9)) * 0.285

    
    mass_0 = np.log10(mass_0)
    mass_funcs_plot(sigma_0, mass_0, F_0, rho_bar_0, rads)
    
    return mass_0

## Calculation of mass function from halo catalog method

In [ ]:
def halo_function(halo_masses, rads, mass_0):
    
    bins = np.linspace(12.5, np.max(halo_masses), len(rads))
    counts, _ = np.histogram(halo_masses, bins=bins)
    bin_widths = np.diff(bins)
    mass = np.sum(counts)
    counts = counts / bin_widths
    counts = counts/1600**3
    
    return counts, bins


## Halo mass fucntion plotting for z-val range

In [ ]:
def halo_mass_plot(files):

    zs = [0, 1, 2, 3, 4]
    rads = np.array([1,2,3,4,5,10, 15, 25,30])
    mass_0 = mass_funcs(rads)


    for i in range(len(files)):
        halo_masses = load_halos(f'{files[i]}')
        halo_masses = np.flip(halo_masses)
        halo_masses = halo_masses/0.695
        halo_masses= np.log10(halo_masses)

        counts, bins = halo_function(halo_masses, rads,mass_0)
        plt.plot(bins[1:], counts, label = f'z={zs[i]}')

    plt.yscale('log')
    plt.xlabel('$log_{10}({M}/{M_{sun}}$)')
    plt.ylabel('$\phi = dn/dlog_{10}M$  $(Mpc/h)^{-3}$')

    plt.legend()
    plt.savefig('mass_func_evo.jpeg', dpi = 200)
    plt.show()

    
halo_files = ['halo_masses.txt', 'halo_masses_1.txt','halo_masses_2.txt', 'halo_masses_3.txt', 'halo_masses_4.txt']  

halo_mass_plot(halo_files)